## Скрипт получения пользователей и их друзей для группы ВКонтакте

In [ ]:
#coding: utf-8
import networkx
from networkx.readwrite import json_graph

import json
%matplotlib inline

In [ ]:
import requests
import json
import time

### Получаем список пользователей группы

In [ ]:
groupID = 77340232  # ID группы (можно получить в строке браузера, зайдя в видеозаписи группы)
offset = 0  # начинаем скачивать участников, начиная с первого
count = 100  # в каждом запросе берем по count записей. Максимум 1000, тут 100 для примера (группа небольшая)

userIDs = []  # список ID пользователей группы

while True:
    print 'Starting offset ' + str(offset)
    
    # делаем запрос к API ВКонтакте
    r = requests.get('https://api.vk.com/method/groups.getMembers?gid=' + str(groupID) + \
                 '&offset=' + str(offset) + '&count=' + str(count))
    
    # если вернулся пустой список с данной страницы, значит мы прошли весь список и пора выходить из цикла
    if len(json.loads(r.text)['response']['users']) == 0:
        break
        
    # добавляем в список ID пользователей группы очередной список ID
    userIDs.extend(json.loads(r.text)['response']['users'])
    offset += count

    # ждем секунду, т. к. API ВКонтакте позволяет этому методу делать ограниченное количество запросов в секунду
    time.sleep(1)

In [ ]:
# проверяем, что выгрузили всех участников группы
print len(userIDs)

### Получаем список друзей каждого участника

In [ ]:
# записываем результат в этот файл
f = open('users.txt', 'w')
f.close()

In [ ]:
userInfo = {}  # словарь userInfo будет содержать список друзей пользователя и его ID

j = 0  # вспомогательная переменная, чтобы отслеживать сколько еще ждать

# для каждого пользователя из нашего списка берем его список друзей
for user in userIDs:
    # используем try-except, чтобы не брать удаленных пользователей и не попасть в бан
    try:
        r = requests.get('https://api.vk.com/method/friends.get?user_id=' + str(user))

        info = json.loads(r.text)['response']

        idsList = []
        for uid in info:
            idsList.append(uid)

        userInfo['id'] = user;
        userInfo['friends'] = idsList

        with open("users.txt", "a") as f:
            f.write(str(userInfo) + '\n')

        if j%100==0:
            print j

        j += 1
        time.sleep(0.5)
    except:
        print 'Waiting...'
        print json.loads(r.text)
        time.sleep(20)